# Regression Model

### Importing Libraries

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import os
os.chdir("drive/MyDrive/DL_Project")

In [2]:
from Dataset import classification_Dataset
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Implementing Model

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define attention mechanism
class AttentionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        attn_weights = self.attention(x)
        return x * attn_weights

# Custom CNN model with ResNet18 and attention layer
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.attention = AttentionLayer(512, 512)  # Assuming ResNet18 output features are 512
        self.fc = nn.Linear(512, num_classes)  # 3 neurons

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.attention(x)  # Apply attention
        x = self.fc(x)  # Final output

        return x.view(-1,3)

model = CNN(3)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CNN(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

### Setting Hyperparameters

In [5]:
num_epochs=50
batch_size=2
learning_rate=0.0001

### Defining Loss Function and Optimizer

In [6]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)

### Training & Validating Model

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [8]:
train_dataset=classification_Dataset("Data/train_set.csv",transform=transform)
train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

In [9]:
valid_dataset=classification_Dataset("Data/validation_set.csv",transform=transform)
valid_dataloader=torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for i, (images, target) in enumerate(train_dataloader):
        images = images.reshape(-1,3,4000,3000).to(device)
        target = target.to(device).view(-1,3)

        optimizer.zero_grad()

        output = model(images)
        output = nn.functional.softmax(output, dim=1)
        loss = criterion(output, target)

        l2_reg_loss = 0.0
        for param in model.parameters():
            l2_reg_loss += torch.norm(param, p=2) ** 2

        loss += 0.05 * l2_reg_loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        avg_loss=total_loss/len(train_dataloader)

    # Performing validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0

        for i, (val_images, val_target) in enumerate(valid_dataloader):
            val_images = val_images.reshape(-1,3,4000,3000).to(device)
            val_target = val_target.to(device).view(-1,3)

            val_output = model(val_images)
            val_output = nn.functional.softmax(val_output, dim=1)
            val_loss += criterion(val_output, val_target).item()


            average_val_loss = val_loss / len(valid_dataloader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.4f}, Validation Loss: {average_val_loss:.4f}')

Epoch [1/50], Training Loss: 370.9625, Validation Loss: 0.9747
Epoch [2/50], Training Loss: 245.4567, Validation Loss: 0.8552


### Saving Model Weights

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))

### Testing Model

In [ ]:
test_dataset=classification_Dataset("Data/test_set.csv",transform=transform)
test_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
correct=[]
predicted=[]
model.eval()
with torch.no_grad():
    test_loss = 0.0

    for i, (test_images, test_target) in enumerate(test_dataloader):
        test_images = test_images.reshape(-1,3,4000,3000).to(device)
        test_target = test_target.to(device).view(-1,3)

        test_output = model(test_images)
        test_output = nn.functional.softmax(test_output, dim=1)
        test_loss += criterion(test_output, test_target).item()

        correct.append(test_target)
        predicted.append(test_output)

        average_test_loss = test_loss / len(test_dataloader)

    print(f'Average Loss: {average_test_loss}')

In [ ]:
for x,y in zip(correct,predicted):
  for i,j in zip(x,y):
    print(i.item(),j.item())